# Advanced Querying Mongo

Importing libraries and setting up connection

In [62]:
!pip3 install pymongo

In [63]:
from pymongo import MongoClient              
client = MongoClient("localhost:27017")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [64]:
# Your Code
db = client.get_database("ironhack")
collection = db.get_collection("companies")
name_companies=collection.find({'name': 'Babelgum'}, {"name": 1, "_id": 0})

In [65]:
list_companies = list(name_companies)
list_companies

[]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [66]:
# Your Code
compani_more_employees = collection.find({'number_of_employees': {'$gt':5000}}).sort('number_of_employees', 1).limit(20)
compani_more_employees = list (compani_more_employees)
compani_more_employees[0:2]

[]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [67]:
# Your Code
compani_founded = collection.find({"$and":[{'founded_year': {'$gte':2000}}, {'founded_year': {'$lte':2005}}]}, {'name': 1, '_id': 0, 'founded_year':1})
list(compani_founded)[0:2]

[]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [68]:
# Your Code
e4 = collection.find({"$and": [{"ipo.valuation_amount": {"$gt": 100000000}},{"founded_year": {"$lt": 2010}}]},{"name":1, "ipo":1, "_id": 0})
list(e4)[0:2]

[]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [69]:
# Your Code
e5 = collection.find({"$and":[{'founded_year': {'$lt':2005}}, {'number_of_employees': {'$lte':1000}}]}).sort('number_of_employees', -1).limit(10)
list(e5)[0:2]

[]

### 6. All the companies that don't include the `partners` field.

In [70]:
# Your Code
e6 = collection.find({'partners' : {"$exists": False}})
list(e6)[0:2]

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [71]:
# Your Code
e7 = collection.find({'name' : {"$regex": ".*\hola.*"} })
list(e7)[0:2]

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [72]:
# Your Code
e8 = collection.find({"$and":[{'number_of_employees': {'$gt':100}}, {'number_of_employees': {'$lt':1000}}]}, {'name':1, 'number_of_employees':1, '_id':0})
list(e8)[0:2]

[]

### 9. Order all the companies by their IPO price in a descending order.

In [73]:
# Your Code
e9 = collection.find({}).sort("ipo.valuation_amount", -1).limit(10)
list(e9)[0:2]

[]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [74]:
# Your Code
e10 = collection.find({}).sort("number_of_employees", -1).limit(10)
list(e10)[0:2]

[]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [75]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [76]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code